# Generate fake requests to local instance

Start local server
* dependencies:
```
docker compose down && docker compose up --build -d
```

* API server
```
cd tests/cli
./run.sh
```


In [3]:
import requests

def generate_sequential_requests(locations: list[tuple[float, float]]):
    """
    Generate sequential requests for the given locations.
    """
    for location in locations:
        requests.get(
            f"http://localhost:8000/nearest_station?lat={location[0]}&lon={location[1]}",
            headers={
                "Content-Type": "application/json",
                "X-API-Key": "123"
            }
        )

In [4]:
import random
import math

EARTH_RADIUS_MILES = 3958.8  # Earth's radius in miles


def _generate_random_location(
        center: tuple[float, float],
        radius_miles_range: tuple[float, float]
) -> tuple[float, float]:
    """
    Generate a random location around the center.
    """
    # Generate random distance within the radius
    random_distance_miles = random.uniform(*radius_miles_range)

    # Generate random bearing (angle) in radians
    random_bearing_rad = 2 * math.pi * random.random()

    # Calculate new latitude and longitude
    center_lat_rad, center_lon_rad = math.radians(center[0]), math.radians(center[1])

    new_lat_rad = math.asin(
        math.sin(center_lat_rad) * math.cos(random_distance_miles / EARTH_RADIUS_MILES) +
        math.cos(center_lat_rad) * math.sin(random_distance_miles / EARTH_RADIUS_MILES) *
        math.cos(random_bearing_rad)
    )
    new_lon_rad = center_lon_rad + math.atan2(
        math.sin(random_bearing_rad) * math.sin(random_distance_miles / EARTH_RADIUS_MILES) *
        math.cos(center_lat_rad),
        math.cos(random_distance_miles / EARTH_RADIUS_MILES) -
        math.sin(center_lat_rad) * math.sin(new_lat_rad)
    )
    return (math.degrees(new_lat_rad), math.degrees(new_lon_rad))


def gerenerate_random_locations(n: int, center: tuple[float, float], radius_miles: float = 10) -> list[tuple[float, float]]:
    """
    Generate n random locations around the center.
    """
    locations = []
    for _ in range(n):
        locations.append(_generate_random_location(center, (0, radius_miles)))
    return locations


def generate_random_outliers(n: int, center: tuple[float, float], radius: float = 10) -> list[tuple[float, float]]:
    """
    Generate n random locations far from the center.
    """
    locations = []
    for _ in range(n):
        locations.append(_generate_random_location(center, (radius, 2 * radius)))
    return locations

In [5]:
# test 1
PHILADELPHIA = (39.9526, -75.1652)
locations = gerenerate_random_locations(10, PHILADELPHIA)
generate_sequential_requests(locations)

# test 2
locations = generate_random_outliers(10, PHILADELPHIA, 50)
generate_sequential_requests(locations)